In [78]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils import data
from sklearn.model_selection import KFold
import scipy.io as scio
import numpy as np

In [79]:
cifar_data = scio.loadmat('cifar10_low_dim.mat')

In [ ]:
# cifar_data['low_dim'][0,0][0,6].shape

## displaying the number of dimension for each scale for scale 0 to 6 (rough to fine scale)
for example: at scale 2, the images' wavelet coefficient on the gMRA ranging from 4 to 50, most of which has 50 coefficient.

In [81]:
low_dim = cifar_data['low_dim']
all_scales = {}
num_scale = low_dim[0][0].shape[1]
for i in range(num_scale):
    scale_i = []
    for j in range(low_dim.shape[1]):
        p = low_dim[0][j][0][i]
        if p.size != 0:
            p = p.reshape(p.shape[0])
        else:
            p = all_scales[i-1][j]
        scale_i.append(p)

    all_scales[i] = np.array(scale_i)
for i in range(7):
    scale_i_sizes = {}
#     size_scale_i = all_scales[i][0].shape[0]
    for j in range(low_dim.shape[1]):
        size_j = all_scales[i][j].shape[0]
        if not scale_i_sizes.get(size_j):
            scale_i_sizes[size_j] = 1
        else:
            scale_i_sizes[size_j] += 1
    print(f"scale {i} num_sizes: {len(scale_i_sizes)}, distribution: {scale_i_sizes}")
#         if all_scales[i][j].shape[0] != size_scale_i:
#             print("point not equal size: ",i,j, all_scales[i][j].shape[0], size_scale_i)


scale 0 num_sizes: 1, distribution: {50: 50000}
scale 1 num_sizes: 1, distribution: {50: 50000}
scale 2 num_sizes: 8, distribution: {50: 46066, 12: 359, 46: 2123, 11: 628, 16: 525, 7: 214, 5: 69, 4: 16}
scale 3 num_sizes: 22, distribution: {50: 35784, 32: 1416, 12: 1074, 28: 933, 8: 381, 16: 308, 10: 1236, 29: 606, 9: 488, 23: 601, 45: 1079, 6: 557, 13: 358, 17: 579, 46: 1600, 11: 214, 4: 187, 5: 371, 7: 502, 31: 433, 40: 888, 21: 405}
scale 4 num_sizes: 39, distribution: {9: 2534, 10: 1992, 11: 1808, 6: 1429, 7: 1704, 8: 2259, 20: 928, 18: 947, 46: 1576, 2: 422, 4: 927, 12: 1480, 16: 592, 5: 1225, 15: 511, 38: 632, 47: 792, 3: 168, 1: 116, 17: 742, 13: 908, 40: 843, 19: 517, 50: 14228, 21: 606, 34: 986, 43: 606, 30: 1227, 24: 832, 32: 1562, 23: 655, 37: 1048, 14: 584, 22: 267, 36: 355, 27: 225, 26: 631, 25: 545, 44: 591}
scale 5 num_sizes: 38, distribution: {6: 5283, 4: 1441, 12: 1906, 21: 1026, 5: 3800, 9: 3354, 3: 169, 7: 5442, 22: 636, 11: 2722, 2: 514, 1: 835, 20: 934, 8: 3698, 13

## Normalizing data
for each scale, backfilling all datapoints so that within each scale, all datapoints are in the same # of dimensions.

In [36]:
max_size_for_scale = {}
for i in range(7):
    size_scale_i = all_scales[i][0].shape[0]
    for j in range(low_dim.shape[1]):
        size_j = all_scales[i][j].shape[0]
        size_scale_i = max(size_j, size_scale_i)
    max_size_for_scale[i] = size_scale_i
print(max_size_for_scale)
for i in range(len(all_scales)):
    embedded_scale_i = []
    for j in range(low_dim.shape[1]):
        embedded_vect = all_scales[i][j]
        if all_scales[i][j].shape[0] < max_size_for_scale[i]:
            embedded_vect = np.zeros((max_size_for_scale[i],))
            embedded_vect[:all_scales[i][j].shape[0]] = all_scales[i][j]
        embedded_scale_i.append(embedded_vect)
    all_scales[i] = np.asarray(embedded_scale_i)
np.save("cifar10_normalized_by_scale", all_scales, allow_pickle=True)

In [44]:
load_data = np.load('cifar10_normalized_by_scale.npy',allow_pickle=True)


In [46]:
cifar_labels = scio.loadmat('cifar_10_labels.mat')
labels = cifar_labels['Labels']
Labels = labels.reshape(labels.shape[0])
print(Labels.shape)

(50000,)


## Classifying with LeNet
images are represented by hyperplane coefficients and classified with LENET


In [73]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (2,2), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (2,2))
        self.fc1   = nn.Linear(16*2, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    def forward(self, x):
        x = x.reshape((8,5,10)).unsqueeze(1)
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, (2,2))
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
class ShallowNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ShallowNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out =self.fc2(out)
        return out

In [76]:
input_size = 50 # low dimension resolution
hidden_size = 20 #
num_classes = 10
num_epochs = 20
batch_size = 8
learning_rate = 10e-3

## Training NN
training the neural network with low dimension image representation


In [90]:
# training network with rough scale representation (scale 0)
scale_i_data = np.matrix(load_data.item()[0])
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(scale_i_data, Labels)
(train_acc, test_acc) = (0,0)
for train_index, test_index in kf.split(scale_i_data,Labels):
#         getting train & test data of k fold & normalize input
    X_train, X_test = scale_i_data[train_index], scale_i_data[test_index]
    X_train -= X_train.min()
    X_train /= X_train.max()
    X_test -= X_test.min()
    X_test /= X_test.max()
    y_train, y_test = Labels[train_index], Labels[test_index]
#         initialize network
    net = ShallowNetwork(input_size, hidden_size, num_classes).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#         convert data to tensors
    X_train_tensor = torch.Tensor(X_train).cuda()
    y_train_tensor = torch.Tensor(y_train).cuda()
    train_dataset = data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = data.DataLoader(dataset=train_dataset,
                          batch_size=batch_size,)
    X_test_tensor = torch.Tensor(X_test).cuda()
    y_test_tensor = torch.Tensor(y_test).cuda()
    test_dataset = data.TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = data.DataLoader(dataset=test_dataset,
                          batch_size=batch_size,)
#         Train network
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = Variable(images).cuda()         
            labels = Variable(labels.long()).cuda()
            optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
            outputs = net(images)                           # Forward pass: compute the output class given a image
            loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
            loss.backward()                                   # Backward pass: compute the weight
            optimizer.step()                                  # Optimizer: update the weights of hidden nodes
#                 if (i+1) % 1000 == 0:                              # Logging
#                     print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
#                  %(epoch+1, num_epochs, i+1, X_train.shape[0]//batch_size, loss.data))
#         Test on test images
    correct = 0
    total = 0
    for images, labels in test_loader:
        labels = labels.long()
        images = Variable(images)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        total += labels.size(0)                    # Increment the total count
        correct += (predicted == labels).sum()     # Increment the correct count
    test_accuracy = 100 * correct / total
#         Test on train images
    correct = 0
    total = 0
    for images, labels in train_loader:
        labels = labels.long()
        images = Variable(images)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        total += labels.size(0)                    # Increment the total count
        correct += (predicted == labels).sum()     # Increment the correct count
    train_accuracy = 100* correct / total
    print(f'Training Accuracy {train_accuracy:.2f}, Testing Accuracy images: {test_accuracy:.2f}')
    train_acc += train_accuracy
    test_acc += test_accuracy
print(f"##############Average k-fold accuracy##############")
print(f"Training Accuracy {(train_acc/10):.2f}, Testing Accuracy images: {(test_acc/10):.2f}")

KeyboardInterrupt: 